### First we gather the data

In [2]:
import scipy

In [3]:
file_path = 'trackedPointsPerFrame'
mattric_struct = scipy.io.loadmat(file_path)
trackedPoints = mattric_struct['trackedPointsPerFrame']
print(type(trackedPoints))
print(trackedPoints)

<class 'numpy.ndarray'>
[[5.31076477e+02 2.40788101e+02 1.00000000e+00]
 [1.29228992e+03 2.86447327e+02 1.00000000e+00]
 [1.25122437e+03 7.30169861e+02 1.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [4]:

# extract a part of the array trackedPointsPerFrame
trackedPointsPerFrame=[]
for i in range(len(trackedPoints)-4):
    trackedPointsPerFrame.append(trackedPoints[:][i:i+5])

print(trackedPointsPerFrame[1])
print(trackedPointsPerFrame[6])
        

[[1.29228992e+03 2.86447327e+02 1.00000000e+00]
 [1.25122437e+03 7.30169861e+02 1.00000000e+00]
 [4.97040863e+02 8.80103821e+02 1.00000000e+00]
 [5.32581970e+02 2.42284805e+02 1.00000000e+00]
 [5.27494141e+02 2.40443817e+02 1.00000000e+00]]
[[1.29007715e+03 2.85938385e+02 1.00000000e+00]
 [1.24919995e+03 7.29873901e+02 1.00000000e+00]
 [4.93769165e+02 8.79642944e+02 1.00000000e+00]
 [5.29032898e+02 2.41920547e+02 1.00000000e+00]
 [5.24253296e+02 2.41078217e+02 1.00000000e+00]]


In [5]:
from Normalization import *
from FindProjectionMatrix import *

In [6]:
#All the points are measured in meters
point_monde_1=[0,0,1]
point_monde_2=[.217,0,1]
point_monde_3=[.222,.118,1]
point_monde_4=[0,.118,1]
point_monde_5=[0,0,1]

points_target=np.array([point_monde_1,point_monde_2,point_monde_3,point_monde_4,point_monde_5])

In [68]:
K=BuildKMatrix()
i=0
for i in range(0,len(trackedPointsPerFrame)-6,6):
    points_source=trackedPointsPerFrame[i]

    #Now we normalize the data
    scaleFactorNormalizationTargetPoints,averageTargetX,averageTargetY=ComputeScalarNormalization(points_target)
    normalizationMatrix_TargetPoints_T=BuildNormalizationMatrix(scaleFactorNormalizationTargetPoints,averageTargetX,averageTargetY)

    scaleFactorNormalizationSourcePoints,averageSourceX,averageSourceY=ComputeScalarNormalization(points_source)
    normalizationMatrix_SourcePoints_T=BuildNormalizationMatrix(scaleFactorNormalizationSourcePoints,averageSourceX,averageSourceY)

    NormalizedPoints_target=MultiplyMatrixArray(normalizationMatrix_TargetPoints_T,points_target)

    NormalizedPoints_source=MultiplyMatrixArray(normalizationMatrix_SourcePoints_T,points_source)
    normalized_Homography=find_homography(NormalizedPoints_source,NormalizedPoints_target)
    #Denormalize 
    Reversed_Tsource=np.linalg.inv(normalizationMatrix_SourcePoints_T)
    reversed_Tsource_mult_NormHomography=np.matmul(Reversed_Tsource,normalized_Homography)
    homography=np.matmul(reversed_Tsource_mult_NormHomography,normalizationMatrix_TargetPoints_T)


    ProjectionMatrix=BuildProjectionMatrix(homography,K)

    FirstPoint=trackedPointsPerFrame[i][0]
    SecondPoint=trackedPointsPerFrame[i][1] 
    ThirdPoint=trackedPointsPerFrame[i][2]
    FourthPoint=trackedPointsPerFrame[i][3] 
    FifthPoint=trackedPointsPerFrame[i][4]



    HFirstPoint=np.matmul(homography,FirstPoint) 
    HSecondPoint=np.matmul(homography,SecondPoint)
    HThirdPoint=np.matmul(homography,ThirdPoint)
    HFourthPoint=np.matmul(homography,FourthPoint)
    HFifthPoint=np.matmul(homography,FifthPoint)


    Subdivision=1
    NormalizedHFirstPoint=HFirstPoint/HFirstPoint[2]
    NormalizedHSecondPoint=HSecondPoint/HSecondPoint[2]
    NormalizedHThirdPoint=HThirdPoint/HThirdPoint[2]
    NormalizedHFourthPoint=HFourthPoint/HFourthPoint[2]
    NormalizedHFifthPoint=HFifthPoint/HFifthPoint[2]

    dim=-500
    NHFirstPointHomo=np.array([NormalizedHFirstPoint[0],NormalizedHFirstPoint[1],dim,1])
    NHSecondPointHomo=np.array([NormalizedHSecondPoint[0],NormalizedHSecondPoint[1],dim,1])
    NHThirdPointHomo=np.array([NormalizedHThirdPoint[0],NormalizedHThirdPoint[1],dim,1])
    NHFourthPointHomo=np.array([NormalizedHFourthPoint[0],NormalizedHFourthPoint[1],dim,1])
    NHFifthPointHomo=np.array([NormalizedHFifthPoint[0],NormalizedHFifthPoint[1],dim,1])

    #promoting the arrays from row to column ones
  

    NHFirstPointHomo=NHFirstPointHomo[:, np.newaxis]
    NHSecondPointHomo=NHSecondPointHomo[:, np.newaxis]
    NHThirdPointHomo=NHThirdPointHomo[:, np.newaxis]
    NHFourthPointHomo=NHFourthPointHomo[:, np.newaxis]
    NHFifthPointHomo=NHFifthPointHomo[:, np.newaxis]



    threeDpoint_1=np.matmul(ProjectionMatrix,NHFirstPointHomo)
    threeDpoint_2=np.matmul(ProjectionMatrix,NHSecondPointHomo)
    threeDpoint_3=np.matmul(ProjectionMatrix,NHThirdPointHomo)
    threeDpoint_4=np.matmul(ProjectionMatrix,NHFourthPointHomo)
    threeDpoint_5=np.matmul(ProjectionMatrix,NHFifthPointHomo)

    X.append([FirstPoint[0], SecondPoint[0], ThirdPoint[0], FourthPoint[0], FifthPoint[0], threeDpoint_1[0], threeDpoint_2[0]/Subdivision, threeDpoint_3[0]/Subdivision  ,threeDpoint_4[0]/Subdivision,threeDpoint_5[0]/Subdivision])
    Y.append([FirstPoint[1], SecondPoint[1] ,ThirdPoint[1] ,FourthPoint[1] ,FifthPoint[1], threeDpoint_1[1]/Subdivision, threeDpoint_2[1]/Subdivision, threeDpoint_3[1]/Subdivision ,threeDpoint_4[1]/Subdivision,threeDpoint_5[1]/Subdivision])

